In [1]:
import cv2
import math
import csv
import hnswlib
from os import path
import numpy as np

# InceptionResNetV2: 1536
# DenseNet121: 1024
# Xception 2048
# ResNet50 2048
# InceptionV3
# VGG16 512
# VGG19 512
# MobileNet
# MobileNetV2
# NASNet: 4032

######################################################
from keras.applications import vgg16
from keras.applications.vgg16 import preprocess_input

feature_dimenstion = 512 
default_tile_size = 224
default_vertical_tiles = 3
default_neighbor_count = 3
######################################################

def prepare_image_paths_labels():
    labels_file = "household_images/labels.csv"
    data = genfromtxt(labels_file, delimiter=',')
    return data


def prepare_image_paths_labels():
    directory_path = "household_images/images"
    labels_file = "household_images/labels.csv"

    with open(labels_file, "rt", encoding="utf8") as f_input:
        csv_input = csv.reader(f_input)
        header = next(csv_input)
        data = [row for row in csv_input]
    
    image_paths = [path.join(directory_path, row[0]) for row in data]
    image_labels = [int(row[1]) for row in data]
    
    return image_paths, image_labels


def open_and_prepare_image(image_path, tile_size = default_tile_size, vertical_tiles = default_vertical_tiles):
    image = cv2.imread(image_path)
    
    target_height = tile_size * vertical_tiles
    scale_percent = target_height/image.shape[0]
    
    horizontal_tiles = image.shape[1] * scale_percent // tile_size
    target_width = tile_size * horizontal_tiles
    
    scale_width = math.floor(scale_percent * image.shape[1])
    
    #print("(scale_width, target_height)", (scale_width, target_height))
    image = cv2.resize(image, (scale_width, target_height), interpolation=cv2.INTER_CUBIC)
    return image


def extract_windows(image, window_size=default_tile_size, channel_count=3, interior_w=False, interior_h=False):
    
    n_cells = (image.shape[0] // window_size, image.shape[1] // window_size)

    if interior_w:
        n_cells = (n_cells[0] - 1, n_cells[1])

    if interior_h:
        n_cells = (n_cells[0], n_cells[1] - 1)

    img_shape = (n_cells[0] * window_size, n_cells[1] * window_size)

    margins = ((image.shape[0] - img_shape[0])//2, (image.shape[1] - img_shape[1])//2)

    windows = np.zeros((n_cells[0] * n_cells[1], window_size, window_size, channel_count))
    coords = np.zeros((n_cells[0] * n_cells[1], 2))

    for i in range(n_cells[0]):
        for j in range(n_cells[1]):
            img = image[(margins[0] + window_size*i):(margins[0] + window_size*(i+1)), (margins[1] + window_size*j):(margins[1] + window_size*(j+1))]
            windows[i * n_cells[1] + j] = img
            coords[i * n_cells[1] + j] = (margins[0] + window_size*i + window_size//2, margins[1] + window_size*j + window_size//2)

    return windows, coords


def convert_output(feats):
    print("feats.shape", feats.shape)
    
    reduced_feats = np.zeros((feats.shape[0]*feats.shape[1]*feats.shape[2], feats.shape[3]))

    index = 0
    for i in range(feats.shape[0]):
        for j in range(feats.shape[1]):
            for k in range(feats.shape[2]):
                reduced_feats[index] = feats[i,j,k,:]
                index += 1

    return reduced_feats


def main():
    
    ######################################################
    model = vgg16.VGG16(weights="imagenet", include_top=False, input_shape=(default_tile_size, default_tile_size, 3))
    ######################################################
    
    image_paths, image_labels = prepare_image_paths_labels()
    
    image_count = len(image_paths)
    neighbor_count = default_neighbor_count
    
    descriptors = []
    labels = []
    indexes = []

    scores = []
    
    window_count = 0
    
    
    for i in range(image_count):
    
        image_path = image_paths[i]
        print(i, image_path)
    
        image = open_and_prepare_image(image_path)
        windows, coords = extract_windows(image)

        window_count = windows.shape[0]

        print("window_count", window_count)
    
        batch = np.zeros((window_count, default_tile_size, default_tile_size, 3))

        for j in range(window_count):
            batch[j] = windows[j]

        x = preprocess_input(batch)
        y = model.predict(x)
        feature = convert_output(y)

        print("feature.shape", feature.shape)
        
        for j in range(feature.shape[0]):
            descriptors.append(feature[j, :])
            labels.append(image_labels[i])
            indexes.append(i)
            
    desc_index = hnswlib.Index(space='cosine', dim=feature_dimenstion)
    desc_index.init_index(max_elements=1000000, ef_construction=500, M=32)
    desc_index.set_ef(500)    
    desc_index.add_items(np.array(descriptors), np.arange(len(descriptors)))
    
    
    for i in range(len(descriptors)):
        descriptor = descriptors[i]
        #print("descriptor.shape", descriptor.shape)
        
        idxs, distances = desc_index.knn_query(descriptor, k=neighbor_count + feature.shape[0])
        #print("idxs.shape", idxs.shape)
        #print("distances.shape", distances.shape)
        
        count = 0
        candidates = 0
        
        for j in range(len(idxs[0])):
            idx = idxs[0][j]
            #print("idx", idx)
            
            if indexes[i] == indexes[idx]:
                continue
                
            candidates += 1
            
            if labels[i] == labels[idx]:
                count += 1
                
            if candidates >= neighbor_count:
                break
        
        #print("Score", count/candidates)
        scores.append(count/candidates)
    
    return sum(scores)/len(scores)


main()

Using TensorFlow backend.


0 household_images/images/IMG_0317.jpg
window_count 12
feats.shape (12, 7, 7, 512)
feature.shape (588, 512)
1 household_images/images/IMG_0332.jpg
window_count 12
feats.shape (12, 7, 7, 512)
feature.shape (588, 512)
2 household_images/images/IMG_0333.jpg
window_count 12
feats.shape (12, 7, 7, 512)
feature.shape (588, 512)
3 household_images/images/IMG_0334.jpg
window_count 12
feats.shape (12, 7, 7, 512)
feature.shape (588, 512)
4 household_images/images/IMG_0452.jpg
window_count 12
feats.shape (12, 7, 7, 512)
feature.shape (588, 512)
5 household_images/images/IMG_0453.jpg
window_count 12
feats.shape (12, 7, 7, 512)
feature.shape (588, 512)
6 household_images/images/IMG_0454.jpg
window_count 12
feats.shape (12, 7, 7, 512)
feature.shape (588, 512)
7 household_images/images/IMG_0456.jpg
window_count 12
feats.shape (12, 7, 7, 512)
feature.shape (588, 512)
8 household_images/images/IMG_0461.jpg
window_count 12
feats.shape (12, 7, 7, 512)
feature.shape (588, 512)
9 household_images/images/IM

0.3110097460616839